## Scraping data from reddit to obtain users emotions on D.C. walkability and traffic

In [ ]:
import praw
import pandas as pd

reddit_read_only = praw.Reddit(client_id="",
							client_secret="",
							user_agent="")


subreddit = reddit_read_only.subreddit("redditdev")

print("Name:", subreddit.display_name)
print("Title:", subreddit.title)
print("Description:", subreddit.description)

In [ ]:
subreddit = reddit_read_only.subreddit("Python")

for post in subreddit.hot(limit=5):
	print(post.title)
	print()